# Create New Cycle

This notebook creates a new active cycle for your Risk Modeling workflow. 

1. Archives the current active cycle (if any)
2. Creates a new cycle from the template
3. Registers stages and steps in the database

**Note:** Only one cycle can be active at a time.

In [1]:
# Import modules directly
from helpers import cycle, database as db, ux
from helpers.context import WorkContext
from helpers.step import Step
from helpers import constants

## Cycle Name

In [2]:
new_cycle_name = ux.text_input("Enter cycle name to proceed", placeholder = constants.CYCLE_NAME_RULES['example'], validation=cycle.validate_cycle_name)
if new_cycle_name is None:
    ux.error("Text input was canceled, execution stopped.")
    sys.exit()
ux.info(f"Cycle name: {new_cycle_name}!")

Enter cycle name to proceed (e.g., Analysis-2025-Q4 OR Analysis-2025-Q4-v1) (enter 'cancel' to stop)


>  Analysis-2021-Q1


## Look into current cycles (if any)

In [3]:
ux.subheader("Current Cycles")

cycles_df = cycle.get_cycle_status()
if not cycles_df.empty:
    ux.dataframe(cycles_df, max_rows=10)
else:
    print("No cycles found")

# Check for active cycle
active_cycle = db.get_active_cycle()
if active_cycle:
    ux.error(f"\nActive cycle exists: {active_cycle['cycle_name']}")

    if ux.yes_no("This will be archived when creating a new cycle. Do you wish to proceed?"):
        ux.warning("Proceeding with archival of {active_cycle['cycle_name']} and creation of ")
    else:
        ux.error("Execution stopped.")
        sys.exit()


else:
    ux.success("No active cycle - ready to create new cycle")

No cycles found


In [14]:
# Creating the cycle
ux.header(f"Creating Cycle: {new_cycle_name}")

# Create the cycle
success = cycle.create_cycle(new_cycle_name)

if success:
    ux.header("CYCLE CREATED SUCCESSFULLY")
    
    # Show the new cycle
    new_cycle = db.get_active_cycle()
    if new_cycle:
        ux.cycle_summary(new_cycle)
    
    # Show directory structure
    from pathlib import Path
    active_dir = Path(f"{constants.WORKFLOWS_PATH}/Active_{new_cycle_name}")
    
    if active_dir.exists():
        ux.subheader("Directory Structure")
        print(f"📁 {active_dir}\n")
        
        for item in sorted(active_dir.iterdir()):
            if item.is_dir():
                print(f"  📂 {item.name}/")
                if item.name == 'notebooks':
                    for subitem in sorted(item.iterdir()):
                        if subitem.is_dir():
                            print(f"      📂 {subitem.name}/")
else:
    ux.error("Failed to create cycle")

0 1
--

Creating cycle: Analysis-2025-Q4
Created directory: /home/jovyan/workspace/workflows/Active_Analysis-2025-Q4
Stage: 1 - Setup
Step: 1 - Initialize
Step: 2 - Validate
0 4
--
1 4
--
2 4
--
3 4
--
Registering stages and steps...
0 2
--
1 2
--
0 3
--
1 3
--
2 3
--
0 2
--
1 2
--
0 4
--
1 4
--
2 4
--
3 4
--
0 2
--
1 2
--
0 4
--
1 4
--
2 4
--
3 4
--
Registered 1 stages with steps

Cycle 'Analysis-2025-Q4' created successfully


Property,Value
Status,ACTIVE
Created,2025-10-15 03:26:52.990430+00:00
Created By,notebook_user


📁 /home/jovyan/workspace/workflows/Active_Analysis-2025-Q4

  📂 files/
  📂 logs/
  📂 notebooks/
      📂 Stage_01_Setup/


In [4]:
# Show directory structure
from pathlib import Path
active_dir = Path(f"/home/jovyan/workspace/workflows/Active_{new_cycle_name}")

if active_dir.exists():
    ux.subheader("Directory Structure")
    print(f"📁 {active_dir}\n")
    
    for item in sorted(active_dir.iterdir()):
        if item.is_dir():
            print(f"  📂 {item.name}/")
            if item.name == 'notebooks':
                for subitem in sorted(item.iterdir()):
                    if subitem.is_dir():
                        print(f"      📂 {subitem.name}/")

In [16]:
ux.header("Verification")

# Check database
active_cycle = db.get_active_cycle()
if active_cycle:
    ux.success(f"Active cycle in database: {active_cycle['cycle_name']}")
else:
    ux.error("No active cycle in database")

# Check filesystem
from pathlib import Path
workflows_path = Path('/home/jovyan/workspace/workflows')
active_dirs = [d for d in workflows_path.iterdir() if d.is_dir() and d.name.startswith('Active_')]

if active_dirs:
    ux.success(f"Active directory found: {active_dirs[0].name}")
else:
    ux.error("No active directory found")

# Show registered steps
if active_cycle:
    progress_df = cycle.get_cycle_progress(active_cycle['cycle_name'])
    if not progress_df.empty:
        ux.subheader(f"The cycle has {len(progress_df)} steps")
        ux.step_progress(progress_df)
    else:
        ux.warning("No stages and steps - template may be empty")

0 1
--


---